In [1]:
import numpy as np
import pandas as pd

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv
from scipy.optimize import minimize
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_2D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta = np.arctan2(x[1], x[0])   
    
    return ndt+rt, theta

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
    term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
    term3 = da(t) - (a(t)/(t-tau)) + kk
    term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q)

        for j in range(1, n):
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_2D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0]**2
    da = lambda t: 0
    ndt = prms[1]
    mu = np.array([prms[2], prms[3]])
    
    fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.01, 
                        dt=0.01, T_max=max(RT))
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta)
            mu_dot_x1 = mu[1]*np.sin(theta)
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt - ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': []}

min_threshold = 0.5
max_threshold = 2.5

min_ndt = 0.1
max_ndt = 1

min_mu = -3.5
max_mu = 3.5

In [6]:
for n in range(150):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    
    RT = []
    Theta = []
    
    print(n+1, end=': ')
    for i in range(250):
        rt, theta = simulate_HSDM_2D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    print('Simulation is done!', end=' / ')
    
    min_ans = minimize(HSDM_2D_likelihood,
                       args=(RT, Theta), 
                       x0=np.array([np.random.uniform(min_threshold, max_threshold),
                                    np.random.uniform(min_ndt, max_ndt), 
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu)]),
                       method='Nelder-Mead', 
                       bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    
    print('Estimation is done!')
    
recovery_df = pd.DataFrame(recovery_df)

1: Simulation is done! / Estimation is done!
2: Simulation is done! / Estimation is done!
3: Simulation is done! / Estimation is done!
4: Simulation is done! / Estimation is done!
5: Simulation is done! / Estimation is done!
6: Simulation is done! / 

/var/folders/n_/r9yj_2js1418_tccsyp_f6s80000gn/T/ipykernel_81432/763051868.py:23: RuntimeWarning: invalid value encountered in log
  log_lik += -np.log(density)


Estimation is done!
7: Simulation is done! / Estimation is done!
8: Simulation is done! / Estimation is done!
9: Simulation is done! / Estimation is done!
10: Simulation is done! / Estimation is done!


In [8]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
0,1.461073,1.538357,0.278904,0.287235,2.910335,3.198200,2.175836,2.313711
1,2.006837,1.926780,0.646058,0.686776,0.969012,0.988291,-2.694441,-2.707988
2,2.345575,2.065546,0.233799,0.335993,2.580243,2.699686,-2.260831,-2.210481
3,2.363057,2.409654,0.406957,0.429422,-2.754203,-2.921447,2.434203,2.703020
4,1.924142,2.463717,0.151691,0.138240,0.212271,0.349692,-2.315371,-3.500000
5,1.413728,1.356252,0.185918,0.239083,-1.199296,-1.238083,-0.576222,-0.532512
6,1.612352,1.429149,0.923928,1.000000,-0.581497,-0.587386,2.314899,2.563623
7,2.083184,2.188494,0.797756,0.760755,-0.782297,-0.852073,1.300216,1.284177
8,0.695894,0.687314,0.790517,0.807982,0.884248,0.775613,-2.453852,-2.463393
9,0.627154,0.612703,0.531848,0.542610,1.422345,1.752835,-1.669178,-1.788742


In [11]:
recovery_df.to_csv('IE_2d_recovery.csv')